# 경사하강법

## 경사하강법 실습

파이토치는 경사하강법을 위한 자동편미분 기능 제공

이 기능을 통해 경사하강법을 구현할 수 있다.

여기서는 경사하강법을 통해 랜덤하게 생성된 텐서가 특정 텐서 값을 근사계산하도록 구현해볼 것. 여기에서 함수의 출력은 목표 텐서와 랜덤 텐서 사이의 차이가 될 것이고 함수의 입력은 랜덤 생성한 텐서의 현재 값이 될 것이다.

In [1]:
import torch
import torch.nn.functional as F

target = torch.FloatTensor([i/10 for i in range(1, 10)])

tensor([0.1000, 0.2000, 0.3000, 0.4000, 0.5000, 0.6000, 0.7000, 0.8000, 0.9000])

랜덤 값을 갖는 텐서를 하나 생성한다. 이 텐서의 requires_grad 속성이 True가 되도록 설정해야 한다.

In [4]:
x = torch.rand_like(target) # x로 미분한 final scalar
x.requires_grad = True
print(x)

tensor([0.6670, 0.4168, 0.7871, 0.0563, 0.0195, 0.4457, 0.2058, 0.4862, 0.3601],
       requires_grad=True)


In [5]:
loss = F.mse_loss(x, target)
loss

tensor(0.1792, grad_fn=<MseLossBackward0>)

while 반복문을 사용하여 두 텐서 값의 차이가 변수 threshold의 값보다 작아질 때까지 미분 및 경사하강법을 반복 수행한다.

In [6]:
threshold = 1e-5
learning_rate = 1.
iter_cnt = 0

while loss > threshold:
    iter_cnt += 1
    loss.backward() # 역전파로 편미분 수행
    x = x - learning_rate * x.grad # learning rate에서 x를 업데이트
    x.detach_()
    x.requires_grad_()

    loss = F.mse_loss(x, target)
    
    print('%d-th Loss : %.4e' % (iter_cnt, loss))
    print(x)

1-th Loss : 1.0841e-01
tensor([0.5410, 0.3686, 0.6789, 0.1327, 0.1263, 0.4800, 0.3156, 0.5559, 0.4801],
       requires_grad=True)
2-th Loss : 6.5579e-02
tensor([0.4430, 0.3311, 0.5947, 0.1921, 0.2093, 0.5067, 0.4010, 0.6102, 0.5734],
       requires_grad=True)
3-th Loss : 3.9671e-02
tensor([0.3668, 0.3020, 0.5292, 0.2383, 0.2739, 0.5274, 0.4675, 0.6524, 0.6460],
       requires_grad=True)
4-th Loss : 2.3999e-02
tensor([0.3075, 0.2793, 0.4783, 0.2742, 0.3242, 0.5435, 0.5191, 0.6852, 0.7024],
       requires_grad=True)
5-th Loss : 1.4518e-02
tensor([0.2614, 0.2617, 0.4386, 0.3022, 0.3632, 0.5561, 0.5593, 0.7107, 0.7463],
       requires_grad=True)
6-th Loss : 8.7824e-03
tensor([0.2255, 0.2480, 0.4078, 0.3239, 0.3936, 0.5658, 0.5906, 0.7305, 0.7805],
       requires_grad=True)
7-th Loss : 5.3128e-03
tensor([0.1976, 0.2373, 0.3839, 0.3408, 0.4173, 0.5734, 0.6149, 0.7460, 0.8070],
       requires_grad=True)
8-th Loss : 3.2139e-03
tensor([0.1759, 0.2290, 0.3652, 0.3540, 0.4356, 0.5793, 0.63

backward 함수를 통해 편미분을 수행한다.

편미분을 통해 얻어진 gradient들이 x_grad에 자동으로 저장되고, 이 값을 활용하여 경사하강법 수행.

backward를 호출하기 위한 텐서의 크기는 **scalar**임에 주의.

손실 값이 점차 줄어드는 것을 볼 수 있고, 텐서 $x$의 값이 목표 텐서 값에 근접해가는 것을 알 수 있다.

학습률 변수를 조정한다면 tensor x가 목표 tensor에 근접해 가는 속도가 달라질 수 있다. 

## 파이토치 autograd

딥러닝은 경사하강법을 통해 학습되므로 미분이 필수. 파이토치는 **오토그래드라는 자동 미분 기능을 제공**한다.

파이토치는 requires_grad 속성이 True인 텐서의 연산을 추적하기 위한 계산 그래프를 구축하고, backward 함수가 호출되면 이 그래프를 따라 미분으로 자동으로 수행하고 계산된 gradient를 채워 놓는다.

다음과 같이 requires_grad 속성을 True로 만들 수 있다.

In [10]:
x = torch.FloatTensor([i for i in range(1,5)]).reshape(2,2).requires_grad_(True)
x

tensor([[1., 2.],
        [3., 4.]], requires_grad=True)

requires_grad 속성이 True인 텐서가 있을 때 이 텐서가 들어간 연산의 결과가 담긴 텐서도 자동으로 requires_grad 속성값을 True로 갖게 된다.

다음 코드와 같이 여러 가지 연산을 수행하였을 때 결과 텐서들은 **모두 requires_grad 속성값을 가지게 된다**.

In [11]:
x1 = x+2
print(x1)

tensor([[3., 4.],
        [5., 6.]], grad_fn=<AddBackward0>)


In [13]:
x2 = x - 2
print(x2)

tensor([[-1.,  0.],
        [ 1.,  2.]], grad_fn=<SubBackward0>)


In [14]:
x3 = x1 * x2
print(x3)

tensor([[-3.,  0.],
        [ 5., 12.]], grad_fn=<MulBackward0>)


In [15]:
y = x.sum()
print(y)

tensor(10., grad_fn=<SumBackward0>)


모든 텐서들이 grad_fn 속성을 갖는다는 점을 주목하자.

예를 들어 텐서 x1이 덧셈 연산의 결과물이기 때문에 x1의 grad_fn 속성이 AddBackward0임을 볼 수 있다.

텐서 y는 sum 함수를 썼으므로 스칼라 값이 되었다. 여기에서 backward 함수를 호출해보자.

In [16]:
y.backward()

그러면 $x, x1, x2, x3, y$ 모두 grad 속성에 gradient 값이 계산되어 저장되었을 것이다.